In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-06-01


Found rundate 2024-05-31 10:00:03 < 2024-06-01 at commit 19bb7d6e; returning commit 00b8e8c


'2024-06-01 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-06-01 10:00:02, NJSP was reporting 250 YTD deaths
Current YTD Deaths (2025-06-01 10:00:03-04:00): 214
Previous year YTD Deaths (adjusted; 2024-06-01 10:00:02): 249.03976667023989
Projected 2025 total: 623.5795419047781
-14.1% change


1ba7858b7dd2b7b2f806deb34d0cc66b6b7f74ad: FAUQStats cache miss: 2024, Sat Jun 01 10:00:02 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,NaN,State Highway 27,1.0,0.0,0.0,1.0,0.0,27,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13201,01,Atlantic,0102,Atlantic City,NaN,State Highway 30 MP 56.5,1.0,1.0,0.0,0.0,0.0,30,0.0,2024-05-26 14:23:00-04:00
13202,11,Mercer,1103,Hamilton Twsp,NaN,State Highway 29 MP .7,1.0,1.0,0.0,0.0,0.0,29,NaN,2024-05-28 01:14:00-04:00
13204,04,Camden,0423,Magnolia Boro,NaN,State Highway 30,1.0,0.0,0.0,1.0,0.0,30,NaN,2024-05-29 09:54:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,9,0,4,0,12
Bergen,7,1,2,0,10
Burlington,9,3,3,0,14
Camden,10,2,6,0,17
Cape May,2,0,0,0,2
Cumberland,4,2,2,0,7
Essex,5,1,8,1,15
Gloucester,7,4,2,0,10
Hudson,2,1,2,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

41.5% through the year, 58.5% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,10,31,9
Bergen,driver,9,14,7
Burlington,driver,14,31,9
Camden,driver,8,19,10
Cape May,driver,1,8,2
...,...,...,...,...
Salem,crashes,5,14,7
Somerset,crashes,5,18,6
Sussex,crashes,2,12,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,41,1,29,1,15
Bergen,26,1,12,2,10
Burlington,37,1,23,4,11
Camden,33,0,22,4,9
Cape May,13,0,12,0,1
Cumberland,25,0,16,9,9
Essex,47,2,17,6,28
Gloucester,33,0,20,12,7
Hudson,18,0,10,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)